In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import os
import sys
import dvc.api
import pandas as pd
import matplotlib as plt

In [3]:
sys.path.append('.')
sys.path.insert(1, '../script/')
import warnings
warnings.filterwarnings('ignore')

In [39]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from train_classifiers import TrainingClassifier

In [67]:
import os
import warnings
import sys
import datetime

import mlflow
import dvc.api
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import pickle

In [5]:
data_file = '/train_store_processed.csv'

# the local data path
local_path = '../data/' + data_file

# the path to the data set
path = 'data/' + data_file

# base repository
repo = '../'

In [6]:
version = 'v7'

# data path using dvc api
data_url = dvc.api.get_url(path = path, repo = repo, rev = version)

# reading the csv file
missing_values = ["n/a", "na", "undefined", '?', 'NA', 'undefined']
train_data = pd.read_csv(data_url, na_values=missing_values)
train_data.head()

,Unnamed: 0,Date,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,DayOfWeek,...,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,PromoInterval,Assortment,StoreType
0,0,2015-07-31,1,20.0,9.0,2008.0,0,13.0,2010.0,5,...,1,1,0,1,2015,7,31,1,0,2
1,1,2015-07-31,2,20.0,11.0,2007.0,1,13.0,2010.0,5,...,1,1,0,1,2015,7,31,1,0,0
2,2,2015-07-31,3,20.0,12.0,2006.0,1,14.0,2011.0,5,...,1,1,0,1,2015,7,31,1,0,0
3,3,2015-07-31,4,20.0,9.0,2009.0,0,1.0,2012.0,5,...,1,1,0,1,2015,7,31,1,2,2
4,4,2015-07-31,5,20.0,4.0,2015.0,0,1.0,2012.0,5,...,1,1,0,1,2015,7,31,1,0,0


In [70]:
mlflow.set_experiment('Sales prediction mlflow')

mlflow.log_param('path', 'path')
mlflow.log_param('version', 'version')
mlflow.log_param('input_rows', train_data.shape[0])
mlflow.log_param('input_cols', train_data.shape[1])

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 21 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Unnamed: 0                 1017209 non-null  int64  
 1   Date                       1017209 non-null  object 
 2   Store                      1017209 non-null  int64  
 3   CompetitionDistance        1017209 non-null  float64
 4   CompetitionOpenSinceMonth  1017209 non-null  float64
 5   CompetitionOpenSinceYear   1017209 non-null  float64
 6   Promo2                     1017209 non-null  int64  
 7   Promo2SinceWeek            1017209 non-null  float64
 8   Promo2SinceYear            1017209 non-null  float64
 9   DayOfWeek                  1017209 non-null  int64  
 10  Sales                      1017209 non-null  float64
 11  Open                       1017209 non-null  int64  
 12  Promo                      1017209 non-null  int64  
 13  StateHoliday

In [30]:
train_data.shape

(844338, 20)

In [8]:
train_data.drop(['StateHoliday'], axis=1, inplace=True)

we have to remove the column where the store is not open and the sales value is 0, since it has no effect.

In [9]:
train_data = train_data[train_data['Open'] == 1]
train_data = train_data[train_data['Sales'] > 0.0]

In [10]:
train_data['Date'] = pd.to_numeric(pd.to_datetime(train_data['Date']))

Then we select the prediction variable and the targe variable as y and x respectively.

In [47]:
# predicted value
y = train_data['Sales']
to_predict = y.head(50000)
X = train_data.drop('Sales',axis=1)
to_train = X.head(50000)
X_train, X_test, y_train, y_test = train_test_split(to_train, to_predict, test_size=0.2, random_state=123)

Here we use sklearn pipeline

In [48]:
data_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])

pipeline = Pipeline(steps = [
               ('preprocessor', data_transformer)
              ,('regressor',RandomForestRegressor(max_depth=64, random_state=0))
           ])

In [49]:
rf_model = pipeline.fit(X_train, y_train)
print(rf_model)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('imputer', SimpleImputer()),
                                 ('scaler', StandardScaler())])),
                ('regressor',
                 RandomForestRegressor(max_depth=64, random_state=0))])


This random forest regressor algorithm is applied on only the first 50,000 rows of the data set.
As the number of data we feed increases the score we get also increases.

In [50]:
rf_model.score(X_test, y_test)

0.8483890556015834

Saving model using time stamp.

In [33]:
path_for_model =  '../models/' + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + '.pkl'
pickle.dump(rf_model, open(path_for_model, 'wb'))

Feature importance for random forest regression model

In [60]:
importances = pipeline.named_steps['regressor'].feature_importances_ 
importances

array([0.04492418, 0.00688575, 0.17575923, 0.        , 0.09897294,
       0.10403739, 0.02044267, 0.08433761, 0.05796963, 0.06825164,
       0.        , 0.1954586 , 0.00583578, 0.        , 0.00102555,
       0.02123492, 0.03391465, 0.0388847 , 0.04206476])